In [1]:
from utils import *
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from collections import deque
import random

# Configure Modeling Parameters and Fetch Data

Enter a ticker and date range you would like to build the model on.  This model takes a a single ticker's data.  Also enter a training size for the proportion of the data you want to include in your training set vs. your test set.

In [3]:
# stock configs
ticker = ['GOOG']
start_date = '2015-04-01'
end_date = '2024-04-05'

# model configs
train_size = 0.8

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 30  # Number of past days we want to use to predict the future.

In [4]:
# Data Fetching
data = fetch_stock_data(ticker, start_date, end_date)[ticker[0]]
data.reset_index(drop=False, inplace=True)
data['Date'] = pd.to_datetime(data['Date']).dt.tz_localize(None)

print(data.shape)
included_days = len(data)
data.head()

(2268, 8)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2015-04-01,27.354897,27.481548,26.901142,27.053724,39261497,0.0,0.0
1,2015-04-02,26.968458,26.968458,26.619267,26.703186,34327989,0.0,0.0
2,2015-04-06,26.538139,26.846792,26.406002,26.764767,26488525,0.0,0.0
3,2015-04-07,26.830338,27.060205,26.726622,26.777481,26057345,0.0,0.0
4,2015-04-08,26.845297,27.118048,26.845297,27.006353,23570536,0.0,0.0


# Neuro-Evolution Model for Stock Trading

This neuro-evolution model is designed to optimize a neural network for making stock trading decisions. The model evolves a population of neural networks over multiple generations to find the fittest network that maximizes investment returns.

## Neural Network Architecture

The neural network used in this model consists of an input layer, a hidden layer, and an output layer. The input layer takes in a state vector representing the price differences over a sliding window. The hidden layer applies a ReLU activation function, and the output layer produces action probabilities using the softmax activation function.

## Evolutionary Process

The evolutionary process follows these steps:

1. **Initialization**: A population of neural networks is initialized with random weights.

2. **Fitness Evaluation**: The fitness of each neural network is evaluated by simulating its trading decisions on historical price data. The fitness is measured as the percentage of investment returns.

3. **Selection**: The population is sorted based on fitness, and the top 40% of the networks are selected as winners. The remaining networks are selected as parents for the next generation based on their fitness probabilities.

4. **Crossover**: Parent networks are paired up, and their weights are crossed over to create child networks. The crossover points are randomly selected for each weight matrix.

5. **Mutation**: The child networks undergo mutation, where a small probability of mutation is applied to each weight. The mutation adds random noise to the weights to introduce diversity in the population.

6. **Replacement**: The winners and mutated child networks form the population for the next generation.

7. **Iteration**: Steps 2-6 are repeated for a specified number of generations or until a satisfactory fitness level is achieved.

## Trading Simulation

The trading simulation is performed using the fittest neural network. The network makes buy and sell decisions based on the current state of the market. The state is represented by a sliding window of price differences.

The network's decisions are as follows:
- **Buy**: If the network outputs a buy signal and there is sufficient funds, a unit of stock is purchased, and the inventory and balance are updated accordingly.
- **Sell**: If the network outputs a sell signal and there is stock in the inventory, a unit of stock is sold, and the balance is updated based on the selling price.
- **Hold**: If the network does not output a buy or sell signal, no action is taken.

The simulation keeps track of the buying and selling states, total gains, and investment percentage.

## Usage

To use the neuro-evolution model:

1. Prepare the historical price data as a list of closing prices.
2. Set the initial parameters, such as the initial money, window size, and skip size.
3. Create an instance of the `NeuroEvolution` class with the desired population size, mutation rate, and other parameters.
4. Call the `evolve` method to start the evolutionary process.
5. Retrieve the fittest neural network after the specified number of generations.
6. Use the fittest network to simulate trading decisions on new price data.

The neuro-evolution model provides an automated approach to optimizing a neural network for stock trading. By evolving a population of networks and selecting the fittest one, the model aims to find a network that can make profitable trading decisions based on historical price patterns.

In [8]:
class NeuralNetwork:
    def __init__(self, id_, hidden_size=128):
        """
        Initialize the neural network.
        
        Args:
            id_ (int): The unique identifier for the network.
            hidden_size (int): The size of the hidden layer (default: 128).
        """
        self.W1 = np.random.randn(window_size, hidden_size) / np.sqrt(window_size)
        self.W2 = np.random.randn(hidden_size, 3) / np.sqrt(hidden_size)
        self.fitness = 0
        self.id = id_

def relu(X):
    """
    Rectified Linear Unit (ReLU) activation function.
    
    Args:
        X (numpy array): Input values.
        
    Returns:
        numpy array: Output values after applying ReLU.
    """
    return np.maximum(X, 0)

def softmax(X):
    """
    Softmax activation function.
    
    Args:
        X (numpy array): Input values.
        
    Returns:
        numpy array: Output values after applying softmax.
    """
    e_x = np.exp(X - np.max(X, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

def feed_forward(X, net):
    """
    Feed-forward computation of the neural network.
    
    Args:
        X (numpy array): Input data.
        net (NeuralNetwork): Neural network object.
        
    Returns:
        numpy array: Output probabilities after feed-forward computation.
    """
    a1 = np.dot(X, net.W1)
    z1 = relu(a1)
    a2 = np.dot(z1, net.W2)
    return softmax(a2)


In [9]:

class NeuroEvolution:
    def __init__(self, population_size, mutation_rate, model_generator,
                 state_size, window_size, trend, skip, initial_money):
        """
        Initialize the NeuroEvolution object.
        
        Args:
            population_size (int): The size of the population.
            mutation_rate (float): The probability of mutation.
            model_generator (callable): A function that generates a new neural network model.
            state_size (int): The size of the state vector.
            window_size (int): The size of the sliding window.
            trend (list): The trend data.
            skip (int): The number of steps to skip in the trend data.
            initial_money (float): The initial amount of money.
        """
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.model_generator = model_generator
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        self.initial_money = initial_money

    def _initialize_population(self):
        """
        Initialize the population of neural networks.
        """
        self.population = []
        for i in range(self.population_size):
            self.population.append(self.model_generator(i))

    def mutate(self, individual, scale=1.0):
        """
        Mutate the weights of a neural network.
        
        Args:
            individual (NeuralNetwork): The neural network to mutate.
            scale (float): The scale of the mutation (default: 1.0).
            
        Returns:
            NeuralNetwork: The mutated neural network.
        """
        mutation_mask = np.random.binomial(1, p=self.mutation_rate, size=individual.W1.shape)
        individual.W1 += np.random.normal(loc=0, scale=scale, size=individual.W1.shape) * mutation_mask
        mutation_mask = np.random.binomial(1, p=self.mutation_rate, size=individual.W2.shape)
        individual.W2 += np.random.normal(loc=0, scale=scale, size=individual.W2.shape) * mutation_mask
        return individual

    def inherit_weights(self, parent, child):
        """
        Inherit weights from a parent network to a child network.
        
        Args:
            parent (NeuralNetwork): The parent neural network.
            child (NeuralNetwork): The child neural network.
            
        Returns:
            NeuralNetwork: The child neural network with inherited weights.
        """
        child.W1 = parent.W1.copy()
        child.W2 = parent.W2.copy()
        return child

    def crossover(self, parent1, parent2):
        """
        Perform crossover between two parent networks to generate two child networks.
        
        Args:
            parent1 (NeuralNetwork): The first parent neural network.
            parent2 (NeuralNetwork): The second parent neural network.
            
        Returns:
            tuple: Two child neural networks resulting from the crossover.
        """
        child1 = self.model_generator((parent1.id+1)*10)
        child1 = self.inherit_weights(parent1, child1)
        child2 = self.model_generator((parent2.id+1)*10)
        child2 = self.inherit_weights(parent2, child2)
        
        # Perform crossover on the first weight matrix
        n_neurons = child1.W1.shape[1]
        cutoff = np.random.randint(0, n_neurons)
        child1.W1[:, cutoff:] = parent2.W1[:, cutoff:].copy()
        child2.W1[:, cutoff:] = parent1.W1[:, cutoff:].copy()
        
        # Perform crossover on the second weight matrix
        n_neurons = child1.W2.shape[1]
        cutoff = np.random.randint(0, n_neurons)
        child1.W2[:, cutoff:] = parent2.W2[:, cutoff:].copy()
        child2.W2[:, cutoff:] = parent1.W2[:, cutoff:].copy()
        
        return child1, child2

    def get_state(self, t):
        """
        Get the state at a given time step.
        
        Args:
            t (int): The time step.
            
        Returns:
            numpy array: The state vector.
        """
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array([res])

    def act(self, p, state):
        """
        Take an action based on the current state.
        
        Args:
            p (NeuralNetwork): The neural network.
            state (numpy array): The current state.
            
        Returns:
            int: The selected action.
        """
        logits = feed_forward(state, p)
        return np.argmax(logits, 1)[0]

    def buy(self, individual):
        """
        Simulate buying and selling based on the decisions of the neural network.
        
        Args:
            individual (NeuralNetwork): The neural network making the decisions.
            
        Returns:
            tuple: The states when buying, states when selling, total gains, and investment percentage.
        """
        initial_money = self.initial_money
        starting_money = initial_money
        state = self.get_state(0)
        inventory = []
        states_sell = []
        states_buy = []

        for t in range(0, len(self.trend) - 1, self.skip):
            action = self.act(individual, state)
            next_state = self.get_state(t + 1)

            if action == 1 and starting_money >= self.trend[t]:
                inventory.append(self.trend[t])
                initial_money -= self.trend[t]
                states_buy.append(t)
                print('day %d: buy 1 unit at price %f, total balance %f' % (t, self.trend[t], initial_money))

            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.trend[t]
                states_sell.append(t)
                try:
                    invest = ((self.trend[t] - bought_price) / bought_price) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell 1 unit at price %f, investment %f %%, total balance %f,'
                    % (t, self.trend[t], invest, initial_money)
                )
            state = next_state

        invest = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        return states_buy, states_sell, total_gains, invest

    def calculate_fitness(self):
        """
        Calculate the fitness of each individual in the population.
        """
        for i in range(self.population_size):
            initial_money = self.initial_money
            starting_money = initial_money
            state = self.get_state(0)
            inventory = []

            for t in range(0, len(self.trend) - 1, self.skip):
                action = self.act(self.population[i], state)
                next_state = self.get_state(t + 1)

                if action == 1 and starting_money >= self.trend[t]:
                    inventory.append(self.trend[t])
                    starting_money -= self.trend[t]

                elif action == 2 and len(inventory):
                    bought_price = inventory.pop(0)
                    starting_money += self.trend[t]

                state = next_state
            invest = ((starting_money - initial_money) / initial_money) * 100
            self.population[i].fitness = invest

    def evolve(self, generations=20, checkpoint=5):
        """
        Evolve the population of neural networks.
        
        Args:
            generations (int): The number of generations to evolve (default: 20).
            checkpoint (int): The number of generations between each checkpoint (default: 5).
            
        Returns:
            NeuralNetwork: The fittest neural network after evolution.
        """
        self._initialize_population()
        n_winners = int(self.population_size * 0.4)
        n_parents = self.population_size - n_winners
        for epoch in range(generations):
            self.calculate_fitness()
            fitnesses = [i.fitness for i in self.population]
            sort_fitness = np.argsort(fitnesses)[::-1]
            self.population = [self.population[i] for i in sort_fitness]
            fittest_individual = self.population[0]
            if (epoch+1) % checkpoint == 0:
                print('epoch %d, fittest individual %d with accuracy %f' % (epoch+1, sort_fitness[0],
                                                                            fittest_individual.fitness))
            next_population = [self.population[i] for i in range(n_winners)]
            total_fitness = np.sum([np.abs(i.fitness) for i in self.population])
            parent_probabilities = [np.abs(i.fitness / total_fitness) for i in self.population]
            parents = np.random.choice(self.population, size=n_parents, p=parent_probabilities, replace=False)
            for i in np.arange(0, len(parents), 2):
                child1, child2 = self.crossover(parents[i], parents[i+1])
                next_population += [self.mutate(child1), self.mutate(child2)]
            self.population = next_population
        return fittest_individual


In [10]:
# Example usage
close = data.Close.values.tolist()
initial_money = 10000
window_size = 30
skip = 1

population_size = 100
generations = 100
mutation_rate = 0.1
neural_evolve = NeuroEvolution(population_size, mutation_rate, NeuralNetwork,
                               window_size, window_size, close, skip, initial_money)

fittest_net = neural_evolve.evolve(generations=generations)

epoch 5, fittest individual 0 with accuracy 14.707793
epoch 10, fittest individual 0 with accuracy 21.762322
epoch 15, fittest individual 0 with accuracy 21.762322
epoch 20, fittest individual 0 with accuracy 21.762322
epoch 25, fittest individual 0 with accuracy 21.762322
epoch 30, fittest individual 0 with accuracy 21.762322
epoch 35, fittest individual 0 with accuracy 21.762322
epoch 40, fittest individual 0 with accuracy 21.762322
epoch 45, fittest individual 0 with accuracy 21.762322
epoch 50, fittest individual 0 with accuracy 21.762322
epoch 55, fittest individual 0 with accuracy 21.762322
epoch 60, fittest individual 0 with accuracy 21.762322
epoch 65, fittest individual 0 with accuracy 21.762322
epoch 70, fittest individual 0 with accuracy 21.762322
epoch 75, fittest individual 0 with accuracy 21.762322
epoch 80, fittest individual 0 with accuracy 21.762322
epoch 85, fittest individual 74 with accuracy 24.676307
epoch 90, fittest individual 0 with accuracy 24.676307
epoch 95, 

In [12]:
states_buy, states_sell, total_gains, invest = neural_evolve.buy(fittest_net)

day 1: buy 1 unit at price 26.703186, total balance 9973.296814
day 4, sell 1 unit at price 27.006353, investment 1.135323 %, total balance 10000.303167,
day 6: buy 1 unit at price 26.926573, total balance 9973.376595
day 7, sell 1 unit at price 26.884687, investment -0.155554 %, total balance 10000.261282,
day 8: buy 1 unit at price 26.446890, total balance 9973.814392
day 11, sell 1 unit at price 26.130758, investment -1.195345 %, total balance 9999.945150,
day 18: buy 1 unit at price 27.684000, total balance 9972.261150
day 22: buy 1 unit at price 27.039000, total balance 9945.222151
day 24: buy 1 unit at price 26.211000, total balance 9919.011150
day 27, sell 1 unit at price 26.785000, investment -3.247364 %, total balance 9945.796150,
day 28, sell 1 unit at price 26.452000, investment -2.170938 %, total balance 9972.248150,
day 30, sell 1 unit at price 26.920000, investment 2.704970 %, total balance 9999.168150,
day 31: buy 1 unit at price 26.692499, total balance 9972.475651
day 

In [13]:
import plotly.graph_objects as go

def visualize_stock_trading(close, states_buy, states_sell, total_gains, invest):
    fig = go.Figure()

    # Plot the closing price
    fig.add_trace(go.Scatter(x=list(range(len(close))), y=close, mode='lines', name='Closing Price', line=dict(color='blue', width=2)))

    # Plot the buying signals
    buy_indices = [i for i in range(len(close)) if i in states_buy]
    buy_prices = [close[i] for i in buy_indices]
    fig.add_trace(go.Scatter(x=buy_indices, y=buy_prices, mode='markers', name='Buying Signal', marker=dict(symbol='triangle-up', size=10, color='green')))

    # Plot the selling signals
    sell_indices = [i for i in range(len(close)) if i in states_sell]
    sell_prices = [close[i] for i in sell_indices]
    fig.add_trace(go.Scatter(x=sell_indices, y=sell_prices, mode='markers', name='Selling Signal', marker=dict(symbol='triangle-down', size=10, color='red')))

    # Customize the layout
    fig.update_layout(
        title=f'Total Gains: {total_gains:.2f}, Total Investment: {invest:.2f}%',
        xaxis_title='Day',
        yaxis_title='Price',
        template='plotly_dark',
        hovermode='x',
        legend=dict(x=0, y=1, orientation='h')
    )

    fig.show()

In [14]:
visualize_stock_trading(data.Close, states_buy, states_sell, total_gains, invest)